In [1]:
r""" Build character sequence-to-sequence training set """
import os
import pandas as pd

with open("movies-sequence-input.txt", "r") as file_input:
    movie_input = file_input.read()
df_input = pd.DataFrame(movie_input.split('\n'),columns=list('i'))
df_input = df_input.fillna(' ')

with open("movies-sequence-output.txt", "r") as file_output:
    movie_output = file_output.read()
df_output = pd.DataFrame(movie_output.split('\n'),columns=list('o'))
df_output = df_output.fillna(' ')

input_texts, target_texts = [], []
input_vocabulary = set()
output_vocabulary = set()
start_token = '\t'
stop_token = '\n'
#max_training_samples = min(25000, len(df_input) - 1)
max_training_samples = min(10000, len(df_input) - 1)


for input_text, target_text in zip(df_input.i, df_output.o):
    target_text = start_token + target_text \
        + stop_token  # <5>
    if(len(input_text)>400):
        input_texts.append(input_text[:400])
    else:
        input_texts.append(input_text)
    if(len(target_text)>400):
        target_texts.append(target_text[:400])
    else:
        target_texts.append(target_text)
    for char in input_text:  # <6>
        if char not in input_vocabulary:
            input_vocabulary.add(char)
    for char in target_text:
        if char not in output_vocabulary:
            output_vocabulary.add(char)


In [2]:
print(df_input.i.str.len().max())

949


In [6]:
input_vocabulary = sorted(input_vocabulary)
output_vocabulary = sorted(output_vocabulary)

input_vocab_size = len(input_vocabulary)
output_vocab_size = len(output_vocabulary)

max_encoder_seq_len = df_input.i.str.len().max()
max_decoder_seq_len = df_output.o.str.len().max()

input_token_index = dict([(char, i) for i, char in enumerate(input_vocabulary)])

target_token_index = dict([(char, i) for i, char in enumerate(output_vocabulary)])

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [5]:
import numpy as np

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, input_vocab_size),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, output_vocab_size),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, output_vocab_size),
    dtype='float32')

for i, (input_text, target_text) in enumerate(
        zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[
            i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[
            i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1


NameError: name 'max_encoder_seq_length' is not defined

In [8]:
print(len(input_texts))
print(max_encoder_seq_length)
print(input_vocab_size)
print(len(input_texts) * max_encoder_seq_length*input_vocab_size)


9562
400
92
351881600


In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

batch_size = 64
epochs = 100
num_neurons = 256

encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(num_neurons, return_sequences=True,
                    return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data, batch_size=batch_size, epochs=epochs,
          validation_split=0.1)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8605 samples, validate on 957 samples
Epoch 1/100
8605/8605 [==============================] - 154s 18ms/step - loss: 0.3874 - acc: 0.0289 - val_loss: 0.3664 - val_acc: 0.0356
Epoch 2/100
8605/8605 [==============================] - 158s 18ms/step - loss: 0.3265 - acc: 0.0366 - val_loss: 0.3224 - val_acc: 0.0439
Epoch 3/100
8605/8605 [==============================] - 161s 19ms/step - loss: 0.2924 - acc: 0.0437 - val_loss: 0.2931 - val_acc: 0.0482
Epoch 4/100
8605/8605 [==============================] - 291s 34ms/step - loss: 0.2734 - acc: 0.0474 - val_loss: 0.2759 - val_acc: 0.0531
Epoch 5/100
8605/8605 [==============================] - 394s 46ms/step - loss: 0.2604 - acc: 0.0505 - val_loss: 0.2669 - val_acc: 0.0546
Epoch 6/100
8605/8605 [==============================] - 163s 19m

In [ ]:
model_structure = model.to_json()
with open("casablancalstmmodel1.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("casablanca_lstm_weights1.h5")

In [88]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [
    Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    output=[decoder_outputs] + decoder_states)


C:\Users\jbark\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


In [89]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense


def instantiate_seq2seq_model(num_encoder_tokens, num_decoder_tokens, num_neurons=256):

    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(num_neurons, return_state=True)
    _, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(num_neurons,))
    decoder_state_input_c = Input(shape=(num_neurons,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model


def construct_seq2seq_model(num_encoder_tokens, num_decoder_tokens, num_neurons=256):

    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(num_neurons, return_state=True)
    _, state_h, state_c = encoder(encoder_inputs)

    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model


In [12]:
max_encoder_seq_len = df_input.i.str.len().max()
# max_encoder_seq_len
# 100
max_decoder_seq_len = df_output.o.str.len().max()


In [13]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [
    Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    output=[decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    thought = encoder_model.predict(input_seq)  # <1>

    target_seq = np.zeros((1, 1, len(output_vocabulary)))  # <2>
    target_seq[0, 0, output_vocabulary.index(stop_token)
        ] = 1.  # <3>
    stop_condition = False
    generated_sequence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + thought) # <4>

        generated_token_idx = np.argmax(output_tokens[0, -1, :])
        generated_char = output_vocabulary[generated_token_idx]
        generated_sequence += generated_char
        if (generated_char == stop_token or
                len(generated_sequence) > max_decoder_seq_len
                ):  # <5>
            stop_condition = True

        target_seq = np.zeros((1, 1, len(output_vocabulary)))  # <6>
        target_seq[0, 0, generated_token_idx] = 1.
        thought = [h, c]  # <7>

    return generated_sequence

def respond(input_text):
    input_text = input_text.lower()
    input_text = ''.join(c if c in input_vocabulary else ' ' for c in input_text)
    input_seq = np.zeros((1, max_encoder_seq_len, len(input_vocabulary)), dtype='float32')
    for t, c in enumerate(input_text):
        input_seq[0, t, input_vocabulary.index(c)] = 1.
    decoded_sentence = decode_sequence(input_seq)
    print('Human: {}'.format(input_text))
    print('Bot:', decoded_sentence)
    return decoded_sentence


C:\Users\jbark\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # This is added back by InteractiveShellApp.init_path()


In [14]:
respond('Hi Rosa, how are you?')
respond('Hi Jim, how are you?')
respond('Hi Barak, how are you?')
respond('Hi Amy, how are you?')
respond('Hi Paris, how are you?')
respond('Hi Joe, how are you?')
respond('Hi Jane, how are you?')
respond('Hey Jane, how are you?')
respond('Hey Jon, how are you?')
respond('Hey John, how are you?')
respond('Hey Joe, how are you?')
respond('Hey Jim, how are you?')
respond('Hey Ashley, how are you?')
respond('Hey my love, how are you?')
respond('Hey Arzu, how are you?')
respond("I'm talking about us.")
respond("What are you trying to say?")


Human: hi rosa, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the sholl crazy?

Human: hi jim, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the shele write she did nothing left is herous again.

Human: hi barak, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the sholl crazy?

Human: hi amy, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the sholl crazy?

Human: hi paris, how are you?
Bot: Well, I can't ready something I want to be happy.

Human: hi joe, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the shele write she did nothing left is herous again.

Human: hi jane, how are you?
Bot: I can't help you that.  What seems looking at the the staming people saying the shele write she did nothing left is herous again.

Human: hey jane, how are you?
Bot: I 

"I don't know.  I don't know.  I was so tarking it all the trub back on the street birthday now and I have no idea...  it to kill off. I love you.\n"

In [50]:
respond('Are you crazy?')

Human: are you crazy?
Bot: I think the say I'm a lot of man and then we have to go to the way they want to go out with anything with you.



"I think the say I'm a lot of man and then we have to go to the way they want to go out with anything with you.\n"

In [20]:
respond('What is perry Como up to?')

Human: what is perry como up to?
Bot: Yes.



'Yes.\n'

In [21]:
respond('It is a beautiful day today.')

Human: it is a beautiful day today.
Bot: I don't know.  I mean the man and I know what I think.



"I don't know.  I mean the man and I know what I think.\n"

In [22]:
respond('Hey Arzu, how are you?')


Human: hey arzu, how are you?
Bot: Y she's bean a break doy to the wedding and I'm not going to have sex what I was ling.



"Y she's bean a break doy to the wedding and I'm not going to have sex what I was ling.\n"